## 1. Imports & Set up

In [1]:
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv
import os
import asyncio
import json
import certifi

from pymongo import MongoClient, UpdateOne
from pymongo.server_api import ServerApi

from datetime import datetime, timedelta
from urllib.request import urlopen

from difflib import SequenceMatcher
import requests

In [2]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_mongodb.vectorstores import MongoDBAtlasVectorSearch
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.messages import SystemMessage, HumanMessage

import asyncio, nest_asyncio
from typing import Dict, List, Any, Optional, Literal, Union
from bson import ObjectId
from collections import defaultdict

from openai import OpenAI, AsyncOpenAI
from langgraph.graph import START, END, StateGraph
from pydantic import BaseModel, Field, field_validator

from IPython.display import Image, display

from tavily import TavilyClient, AsyncTavilyClient

In [3]:
load_dotenv(".env", override=True)

OpenAI_api_key = os.getenv("OpenAI_api_key")
llm_model = "gpt-5-nano"
embed = OpenAIEmbeddings(model="text-embedding-3-small", api_key=OpenAI_api_key)
llm_client = OpenAI(api_key=OpenAI_api_key)

financialmodellingprep_api_key = os.getenv("financialmodellingprep_api_key")

tavily_api_key = os.getenv("tavily_api_key")

## 2. Input Handling & Global State Set up

In [4]:
class InputTicker_State(BaseModel):
    ticker: str = Field(description="The stock ticker symbol, e.g., AAPL for Apple Inc.")
    name: str = Field(description="The name of the company associated with the ticker symbol.")
    exchange: Optional[str] = Field(description="The stock exchange where the ticker is listed, e.g., NASDAQ, NYSE.")
    one_weekb4_today: str = Field(description = "yyyy-mm-dd date that is 1 week b4 today to get latest articles")

    tavily_article_list: Optional[List[Dict]] = Field(description="stores articles from tavily extraction")
    useful_articles_list: Optional[List[Dict]] = Field(description="stores articles as it flows thru the graph")

    news_answer: str = Field("Summarised final news to give to the end user")
    bullish_act: Any = Field("What to do to profit from this information")
    bearish_act: Any = Field("What to do to prevent bad events from affecting my stocks")

In [5]:
def get_jsonparsed_data(url):
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)

def handle_ticker_input(state: InputTicker_State) -> Dict[str, Any]:
    search_ticker_endpoint = (
        "https://financialmodelingprep.com/stable/search-symbol?query=" + 
        state.ticker + 
        "&apikey=" + financialmodellingprep_api_key + 
        "&limit=1"
    )
    
    try:
        result = get_jsonparsed_data(search_ticker_endpoint)
        
        # Return a dictionary with the fields to update
        return {
            "ticker": result[0]['symbol'],
            "name": result[0]['name'],
            "exchange": result[0].get('exchange', None)
        }
    
    except IndexError as ie: 
        print("Please key in a valid ticker")
        raise ValueError(f"Invalid ticker: {state.ticker}")
    except Exception as e:
        print(f"Error handling ticker input for {state.ticker}: {e}")
        raise e

## 3. News Search Section

In [6]:
async def search_query(query: str, tavily_client):
    """Run a single Tavily search query asynchronously
    tavily_client.search() is a synchronous blocking function (it's not actually async, even though you wrote async def)
    """
    result = await tavily_client.search(
        query=query,
        max_results=4,
        topic="news",
        search_depth="advanced",
        days=7,
        include_raw_content=True,
        exclude_domains=["linkedin.com", "youtube.com", "wikipedia.com", "facebook.com"]
    )

    return result.get("results", [])

In [7]:
def deduplicate_aggressive(articles: list[dict], min_score: float = 0.3) -> list[dict]:
    """
    More aggressive deduplication using multiple criteria
    """
    unique_articles = []
    seen_urls = set()
    seen_titles = set()
    
    for article in articles:
        # exclude articles with bad score < 0.3
        if article.get('score', min_score) < min_score:
            continue

        url = article.get('url', '')
        title = article.get('title', '').lower().strip()
        title = title[:title.rfind("-")].strip()
        
        # Skip if URL or exact title already seen
        if url in seen_urls or title in seen_titles:
            continue
        
        # Check for very similar titles (fuzzy matching)
        is_similar = False
        for seen_title in seen_titles:
            similarity = SequenceMatcher(None, title, seen_title).ratio()
            if similarity > 0.75:  # 75% similar
                is_similar = True
                break
        
        if not is_similar:
            unique_articles.append(article)
            seen_urls.add(url)
            seen_titles.add(title)
    
    return unique_articles

In [8]:
async def run_parallel_searches(state: InputTicker_State) -> Dict[str, Any]:
    # Initialize async client
    tavily_client = AsyncTavilyClient(api_key=tavily_api_key)

    """Run both queries in parallel"""
    query1 = "semiconductor OR AI chips"
    query2 = "HBM OR CoWoS OR export controls OR ASML OR TSMC OR NVIDIA OR memory pricing OR PC demand OR smartphone demand"
    
    print("Trying to extract news from online sites using tavily")
    # Run both queries concurrently
    results = await asyncio.gather(
        *(tavily_client.search(
            query=q,
            max_results=2,
            topic="news",
            search_depth="advanced",
            days=7,
            include_raw_content=True,
            exclude_domains=["linkedin.com", "youtube.com", "wikipedia.com", "facebook.com"]
            )
        for q in [query1, query2]
        ) 
    )
    
    articles_query1 = results[0].get("results", [])
    articles_query2 = results[1].get("results", [])
    
    # Combine results if needed
    all_articles = deduplicate_aggressive(articles_query1 + articles_query2)
    print(all_articles)
    print("Extracted news from online sites using tavily")
    
    return {"tavily_article_list" : all_articles}

In [10]:
SYSTEM_PROMPT = """You are a financial-news traige assistant related to semiconductor market.
Use ONLY the provided article content and metadata. Do not add external facts.
If the content is paywalled, boilerplate, or not substantively about the subject, set keep=false.

Output MUST be valid JSON matching the schema.
"""

def build_user_prompt(article: Dict[str, Any], subject_hint: Optional[str] = None) -> str:
    """
    subject_hint: optional (e.g., ticker name) to help the LLM judge relevance when extending project to 1 stock.
    """
    url = article.get("url", "")
    title = article.get("title", "")
    published_date = article.get("published_date", "")
    score = article.get("score", None)
    content = (article.get("content") or "").strip()

    # Keep prompts stable: truncate very long pages
    if len(content) > 14000:
        content = content[:11000] + "\n...\n" + content[-2500:]

    hint_line = f"Subject hint: {subject_hint}\n" if subject_hint else ""

    return f"""{hint_line} Metadata:
        - url: {url}
        - title: {title}
        - published_date: {published_date}
        - score: {score}

        Task:
        Read the article content and produce JSON fields as per the schema:
        - Decide keep or not, keep_score and keep_reason
        keep_reason must be of type string
        - Identify primary_topic, topics
        - Determine stance used only these values - ("bullish", "bearish", "neutral", "unclear")
        - Determine time_horizon of the impact based on claims, use only these values - ("short", "medium", "long") where short is <1Year, medium is 1-3 years and long is >3 years
        For stance & time_horizon, if unsure simply return unclear
        - Write text_for_rag (information-dense; preserve numbers/dates/entities)
        - Write summary (2–6 sentences)
        Note that text_for_rag and summary fields must be of type string

        Article content:
        \"\"\"{content}\"\"\"
        """

In [11]:
async def analyse_one_article_llm(
    llm_client,
    article: Dict[str, Any],
    llm_model: str,
    subject_hint: Optional[str] = None,
    max_retries: int = 1
) -> Dict[str, Any]:
    """
    Returns a new dict:
      {
        ...original article fields...,
        "llm": <ArticleLLMFields as dict>
      }
    Raises if the LLM response can't be validated after retries (handled by caller).
    """
    for attempt in range(max_retries + 1):
        try:
            prompt = build_user_prompt(article, subject_hint=subject_hint)

            resp = llm_client.chat.completions.create(
                model=llm_model,
                messages=[
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": prompt},
                ],
            )

            raw = (resp.choices[0].message.content or "").strip()

            # Validate structured output
            parsed = ArticleLLMFields.model_validate_json(raw)

            article_fields = ArticleFields(
                news_url=article.get("url", "No url"), 
                title=article.get("title", "No title"),
                score=article.get("score", 0.0),
                published_date=article.get("published_date", "No date"),
                content=article.get("content", "No content"),
                llm_output=parsed
            )
            
            return article_fields.model_dump()
        
        # handle failure logic - by retrying twice
        except Exception as e:
            if attempt < max_retries:
                print(f"Attempt {attempt + 1} failed for {article.get('title', 'Unknown title')} due to {e}")
                await asyncio.sleep(1.5)
            else:
                print(f"Final attempt failed for {article.get('title', 'Unknown title')}")
                return {}


In [12]:
async def process_all_articles_node(state: InputTicker_State) -> Dict[str, Any]:
    """
    LangGraph node that processes all articles with LLM analysis.
    
    Expected state keys:
        - articles: List[Dict[str, Any]] - processed articles from Tavily + LLM output
        - client: OpenAI client
        - model: str - model name
        - subject_hint: Optional[str] - hint for analysis
    
    Returns updated state with:
        - processed_articles: List[ArticleFields] - successfully processed articles
    """
    articles = state.tavily_article_list
    
    print(f"Processing {len(articles)} articles...")
    
    # Process all articles concurrently with retry logic
    tasks = [
        analyse_one_article_llm(
            llm_client= llm_client,
            article= article,
            llm_model= llm_model
        )
        for article in articles
    ]
    
    results = await asyncio.gather(*tasks)
    
    # Separate successful and failed articles
    processed_articles = []
    failed_articles = []
    
    for i, result in enumerate(results):
        if result != {}:
            processed_articles.append(result)
        else:
            failed_articles.append(articles[i])
    
    print(f"Successfully processed: {len(processed_articles)}/{len(articles)}")
    print(f"Failed: {len(failed_articles)}/{len(articles)}")
    
    return {
        "useful_articles_list": processed_articles,
    }

In [29]:
def synthesise_semicon_news_node(state: InputTicker_State) -> InputTicker_State:
    query = (
        "Latest semiconductor industry news including related to AI chips, memory pricing, "
        "NVDA, TSMC, ASML. Do let me know how I should use this information in stock trading"
    )
    articles = state.useful_articles_list

    kept = [a for a in articles if a.get("llm_output").get("keep") is True]
    kept.sort(key=lambda x: x.get("llm_output").get("keep_score", 0.0), reverse=True)

    top_k = kept[:3]
    print(top_k)

    evidence_lines = []
    for index, article in enumerate(top_k, start=1):
        llm_article = article.get("llm_output") or {}

        primary_topic = llm_article.get("primary_topic", "")
        stance =  llm_article.get("stance", "unclear")
        horizon = llm_article.get("time_horizon", "unclear")  # your schema uses time_horizon
        summary = llm_article.get("summary", "")

        evidence_lines.append(
            f"[{index}] topic={primary_topic} | stance={stance}/{horizon}\n"
            f"summary: {summary}"
        )

    evidence = "\n\n".join(evidence_lines)

    response = llm_client.chat.completions.create(
        model=llm_model,
        messages=[
            {
                "role": "system",
                "content": (
                    "I am a finance news synthesiser. I only use the provided article evidence for facts. "
                    "I provide general risk-management and idea-generation guidance for stock trading for RETAIL TRADERS! "
                    "I keep it concise and structured. "
                    "Return ONLY valid JSON matching the schema."
                ),
            },
            {
                "role": "user",
                "content": f"""Query: {query}

                    Article evidence (each item is a separate article; treat as potentially overlapping):
                    {evidence}

                    Task:
                    1) YOU MUST produce JSON fields as per the schema to answer the query in 2–4 sentences, synthesising the main themes across the evidence.
                    2) Give actionables to protect against stock shocks: 3–4 bullet points.
                    3) Give potential opportunities: 3–4 bullet points.

                    Constraints:
                    - Do not invent facts not supported by the evidence.
                    - If evidence is mixed/unclear, say so plainly.
                    - Bullets must be concise and practical.

                    Output Schema (JSON only):
                    - news_answer: string (covers the latest important news)
                    - bullish_act: string (actionables to capitalise on these trends)
                    - bearish_act: string (pitfalls / bearish news to be aware of)
                    """,
            },
        ],
    )

    raw = (response.choices[0].message.content or "").strip()

    # Validate structured output (raw is a JSON string, so parse it)
    try:
        parsed = json.loads(raw)
    except json.JSONDecodeError:
        # Fallback: keep pipeline alive even if the model breaks format
        return {
            "news_answer": "I do not have enough high-confidence articles to answer this query.",
            "bearish_act": "I do not have enough high-confidence articles to answer this query.",
            "bullish_act": "I do not have enough high-confidence articles to answer this query.",
        }

    return {
        "news_answer": parsed.get("news_answer", ""),
        "bullish_act": parsed.get("bullish_act", ""),
        "bearish_act": parsed.get("bearish_act", ""),
    }

## 4. Graph Initialisation & Post workflow Handling

In [ ]:
user_input = 'NVDA'

workflow = StateGraph(InputTicker_State)
workflow.add_node("user_ticker_input", handle_ticker_input)
workflow.add_node("Internet Search Tool", run_parallel_searches)
workflow.add_node("llm_output_article", process_all_articles_node)
workflow.add_node("llm_generate_ArticleReport", synthesise_semicon_news_node)

# Add edges
workflow.add_edge(START, "user_ticker_input")
workflow.add_edge("user_ticker_input", "Internet Search Tool")
workflow.add_edge("Internet Search Tool", "llm_output_article")
workflow.add_edge("llm_output_article", "llm_generate_ArticleReport")
workflow.add_edge("llm_generate_ArticleReport", END)

# Compile the graph
graph = workflow.compile(checkpointer=MemorySaver())

# Create config
config = {"configurable": {"thread_id": "1"}}

# inititalise start state = 
OneWeek_b4 = datetime.now() - timedelta(days = 7)
OneWeek_b4_today = str(OneWeek_b4.year) + "-" + str(OneWeek_b4.month) + "-" + str(OneWeek_b4.day)
initial_state = InputTicker_State(
    ticker=user_input,
    name="",
    exchange=None,
    one_weekb4_today=OneWeek_b4_today,
    tavily_article_list=None,
    useful_articles_list=None
)

# Invoke the graph with initial state
result = await graph.ainvoke(initial_state, config)

[{'news_url': 'https://www.marketwatch.com/story/these-overlooked-stocks-are-a-backdoor-way-to-play-the-ai-chip-boom-7157eab9', 'title': 'These overlooked stocks are a backdoor way to play the AI chip boom - MarketWatch', 'score': 0.62495816, 'published_date': 'Sat, 31 Jan 2026 13:30:00 GMT', 'content': '### Not without risk\n\nDeutsche Bank’s Weathers suggested investors temper their enthusiasm around the AI opportunity. The volume of advanced chips for AI make up a small share of the broader semiconductor market, she told MarketWatch.\n\nEven with the memory market’s newfound momentum, chip fabs are at capacity, she explained, meaning there will be limited opportunities to increase wafer production until more clean-room space is added — and that isn’t expected to happen until at least 2027.\n\nAt a high level, Weathers said near-term growth for materials players will be tied more to this cyclical recovery in analog chips. [...] One of Qnity’s most important products is its advanced t

In [35]:
result

{'ticker': 'NVDA',
 'name': 'NVIDIA Corporation',
 'exchange': 'NASDAQ',
 'one_weekb4_today': '2026-1-26',
 'tavily_article_list': [{'url': 'https://www.marketwatch.com/story/these-overlooked-stocks-are-a-backdoor-way-to-play-the-ai-chip-boom-7157eab9',
   'title': 'These overlooked stocks are a backdoor way to play the AI chip boom - MarketWatch',
   'score': 0.62495816,
   'published_date': 'Sat, 31 Jan 2026 13:30:00 GMT',
   'content': '### Not without risk\n\nDeutsche Bank’s Weathers suggested investors temper their enthusiasm around the AI opportunity. The volume of advanced chips for AI make up a small share of the broader semiconductor market, she told MarketWatch.\n\nEven with the memory market’s newfound momentum, chip fabs are at capacity, she explained, meaning there will be limited opportunities to increase wafer production until more clean-room space is added — and that isn’t expected to happen until at least 2027.\n\nAt a high level, Weathers said near-term growth for mat